Created 09/01/2026 by truji@mit.edu

A simple candidate filter for picking targets. 
Takes observatory code and list of possible targets from, in my case, the NEA Observation Planning Aid, and narrows them down based on observing window etc.
Uses JPL Horizons to do this.

In [1]:
from astroquery.jplhorizons import Horizons
from astropy import units as u
from astropy.coordinates import AltAz, EarthLocation, SkyCoord, get_body, get_sun
from astropy.time import Time
import re
from tqdm import tqdm

In [2]:
raw = r"""(164400)     Pos = (10 01, -60.7), V = 15.6, El. =  90.9, 1-sig unc. =      0.1", b = -04.4, Mot. =   2.090 deg/day, Apollo, brightening

  (35107)     Pos = (06 45, -13.9), V = 15.7, El. = 143.0, 1-sig unc. =      0.0", b = -07.6, Mot. =   1.028 deg/day, PHA, Apollo

   (2212)     Pos = (05 05, +41.9), V = 16.1, El. = 145.6, 1-sig unc. =      0.0", b = +00.4, Mot. =   0.580 deg/day, Apollo

 (836372)     Pos = (07 41, +02.0), V = 16.3, El. = 159.2, 1-sig unc. =      0.0", b = +11.9, Mot. =   2.025 deg/day, Amor, fading

 (500080)     Pos = (06 36, +05.5), V = 16.4, El. = 160.3, 1-sig unc. =      0.0", b = -00.9, Mot. =   0.414 deg/day, PHA, Apollo, brightening

   (1943)     Pos = (09 54, -07.8), V = 16.5, El. = 131.8, 1-sig unc. =      0.0", b = +34.7, Mot. =   0.417 deg/day, Amor, brightening

 (367390)     Pos = (05 15, -06.0), V = 16.6, El. = 138.5, 1-sig unc. =      0.0", b = -24.1, Mot. =   3.105 deg/day, Apollo, fading

 (137802)     Pos = (07 05, +06.5), V = 16.8, El. = 163.9, 1-sig unc. =      0.0", b = +06.0, Mot. =   1.172 deg/day, Amor, fading

 (533722)     Pos = (09 22, +31.3), V = 16.9, El. = 151.4, 1-sig unc. =      0.0", b = +44.5, Mot. =   0.807 deg/day, PHA, Apollo, fading

  (96744)     Pos = (06 12, -49.3), V = 17.0, El. = 106.9, 1-sig unc. =      0.0", b = -26.5, Mot. =   0.642 deg/day, Apollo, brightening

 2021 CM9     Pos = (07 32, -12.4), V = 17.1, El. = 145.3, 1-sig unc. =      0.0", b = +03.3, Mot. =   1.576 deg/day, Amor, brightening

 (549328)     Pos = (12 02, +35.2), V = 17.1, El. = 117.9, 1-sig unc. =      0.0", b = +76.8, Mot. =   1.074 deg/day, Apollo

  (26663)     Pos = (14 09, +00.7), V = 17.1, El. =  78.9, 1-sig unc. =      0.1", b = +57.7, Mot. =   4.153 deg/day, PHA, Apollo, fading

  (85628)     Pos = (13 44, +32.4), V = 17.2, El. =  96.8, 1-sig unc. =      0.0", b = +77.4, Mot. =   0.892 deg/day, Amor, fading

   (4257)     Pos = (13 54, -70.3), V = 17.3, El. =  66.3, 1-sig unc. =      0.0", b = -08.0, Mot. =   0.590 deg/day, Apollo, brightening

  (96536)     Pos = (07 29, -73.9), V = 17.3, El. =  83.9, 1-sig unc. =      0.1", b = -23.4, Mot. =   0.910 deg/day, Apollo, brightening

  (11398)     Pos = (08 45, +00.8), V = 17.4, El. = 150.2, 1-sig unc. =      0.0", b = +25.4, Mot. =   0.211 deg/day, Amor, brightening

 (152931)     Pos = (15 01, +63.2), V = 17.4, El. =  99.1, 1-sig unc. =      0.0", b = +48.2, Mot. =   0.644 deg/day, Aten, fading

 (338049)     Pos = (04 17, -35.2), V = 17.5, El. = 108.2, 1-sig unc. =      0.1", b = -45.7, Mot. =   2.225 deg/day, Amor

 (496818)     Pos = (07 22, -07.0), V = 17.5, El. = 150.8, 1-sig unc. =      0.0", b = +03.5, Mot. =   0.554 deg/day, Amor, fading

 (207970)     Pos = (04 04, +08.2), V = 17.7, El. = 131.0, 1-sig unc. =      0.0", b = -31.6, Mot. =   0.635 deg/day, Amor, fading

 (419922)     Pos = (03 54, -07.6), V = 17.7, El. = 121.6, 1-sig unc. =      0.0", b = -42.5, Mot. =   1.334 deg/day, Apollo, fading

 (259221)     Pos = (11 45, +41.6), V = 17.8, El. = 121.9, 1-sig unc. =      0.4", b = +70.1, Mot. =   4.347 deg/day, Apollo, brightening

 2002 XT90    Pos = (08 46, -10.4), V = 17.8, El. = 141.1, 1-sig unc. =      0.0", b = +19.8, Mot. =   2.626 deg/day, Apollo, fading

 (163051)     Pos = (11 09, -14.2), V = 17.8, El. = 113.1, 1-sig unc. =      0.0", b = +41.7, Mot. =   0.905 deg/day, PHA, Apollo, brightening

  (52340)     Pos = (00 25, +38.8), V = 17.8, El. =  93.8, 1-sig unc. =      0.1", b = -23.8, Mot. =   0.441 deg/day, Apollo, brightening

 (843079)     Pos = (06 05, +14.0), V = 17.8, El. = 160.6, 1-sig unc. =      0.0", b = -03.6, Mot. =   0.362 deg/day, Amor, fading

   (5131)     Pos = (05 19, +00.9), V = 17.8, El. = 143.7, 1-sig unc. =      0.0", b = -20.1, Mot. =   0.523 deg/day, Apollo, fading

 (164206)     Pos = (09 08, -65.7), V = 17.9, El. =  89.8, 1-sig unc. =      0.2", b = -12.1, Mot. =   0.272 deg/day, Amor, fading

 (244670)     Pos = (03 40, -27.4), V = 17.9, El. = 107.5, 1-sig unc. =      0.0", b = -52.6, Mot. =   0.417 deg/day, Apollo, brightening

 2000 BH19    Pos = (10 41, +15.5), V = 17.9, El. = 132.2, 1-sig unc. =      0.0", b = +57.5, Mot. =   1.192 deg/day, Amor

 2004 MO3     Pos = (09 02, -09.3), V = 18.0, El. = 139.8, 1-sig unc. =      0.1", b = +23.6, Mot. =   3.445 deg/day, Apollo, fading

 (198856)     Pos = (14 20, +41.6), V = 18.0, El. =  93.9, 1-sig unc. =      0.0", b = +66.7, Mot. =   0.714 deg/day, Amor, fading

 (141765)     Pos = (08 40, +38.6), V = 18.0, El. = 156.2, 1-sig unc. =      0.0", b = +37.1, Mot. =   0.649 deg/day, Amor, fading

  (89830)     Pos = (14 41, +57.5), V = 18.0, El. =  98.4, 1-sig unc. =      0.0", b = +54.0, Mot. =   0.205 deg/day, PHA, Amor, fading

  (25330)     Pos = (14 56, +10.5), V = 18.0, El. =  72.3, 1-sig unc. =      0.0", b = +56.3, Mot. =   0.990 deg/day, Apollo, fading

   (4055)     Pos = (04 17, -16.4), V = 18.1, El. = 121.0, 1-sig unc. =      0.0", b = -41.3, Mot. =   0.294 deg/day, Amor, fading

 (859642)     Pos = (05 56, +07.7), V = 18.1, El. = 155.3, 1-sig unc. =      0.0", b = -08.6, Mot. =   0.623 deg/day, Amor, fading

 (847304)     Pos = (04 15, +20.9), V = 18.1, El. = 137.3, 1-sig unc. =      0.0", b = -21.2, Mot. =   1.148 deg/day, Amor, fading

  (24475)     Pos = (14 02, +11.2), V = 18.2, El. =  84.6, 1-sig unc. =      0.0", b = +67.0, Mot. =   0.691 deg/day, Amor

   (1685)     Pos = (14 24, -23.5), V = 18.2, El. =  67.2, 1-sig unc. =      0.0", b = +34.6, Mot. =   0.418 deg/day, Apollo

 2025 XV8     Pos = (05 22, +22.0), V = 18.3, El. = 152.7, 1-sig unc. =      0.1", b = -08.3, Mot. =   4.957 deg/day, 1-opp Apollo, fading

 (137078)     Pos = (06 08, +57.1), V = 18.4, El. = 142.7, 1-sig unc. =      0.0", b = +17.0, Mot. =   0.567 deg/day, Apollo, fading

  (89355)     Pos = (14 53, +06.9), V = 18.4, El. =  71.4, 1-sig unc. =      0.0", b = +54.8, Mot. =   0.942 deg/day, Amor

 (137084)     Pos = (11 54, -37.3), V = 18.4, El. =  92.3, 1-sig unc. =      0.0", b = +24.2, Mot. =   1.065 deg/day, Apollo, brightening

 (866876)     Pos = (02 52, -16.4), V = 18.4, El. = 104.1, 1-sig unc. =      0.0", b = -60.0, Mot. =   0.370 deg/day, PHA, Apollo

 (678861)     Pos = (01 06, -45.0), V = 18.4, El. =  72.2, 1-sig unc. =      0.0", b = -71.9, Mot. =   1.635 deg/day, Aten

 (612522)     Pos = (13 58, -11.1), V = 18.5, El. =  77.0, 1-sig unc. =      0.0", b = +48.5, Mot. =   1.559 deg/day, PHA, Apollo, fading

 (523604)     Pos = (10 46, +03.9), V = 18.5, El. = 126.9, 1-sig unc. =      0.0", b = +52.0, Mot. =   0.228 deg/day, Amor

  (21088)     Pos = (12 21, -41.3), V = 18.5, El. =  85.8, 1-sig unc. =      0.0", b = +21.3, Mot. =   0.496 deg/day, Amor, brightening

 2011 OK5     Pos = (07 20, +51.9), V = 18.5, El. = 150.3, 1-sig unc. =      0.0", b = +25.3, Mot. =   1.222 deg/day, Amor, brightening

 2025 SQ3     Pos = (06 44, -00.3), V = 18.5, El. = 156.0, 1-sig unc. =      0.0", b = -01.7, Mot. =   0.594 deg/day, 1-opp Amor, fading

 (162635)     Pos = (12 28, -06.0), V = 18.5, El. =  99.6, 1-sig unc. =      0.0", b = +56.4, Mot. =   0.511 deg/day, Amor

 (138013)     Pos = (08 08, -03.0), V = 18.6, El. = 152.2, 1-sig unc. =      0.0", b = +15.4, Mot. =   0.357 deg/day, Apollo, brightening

 (312473)     Pos = (03 21, -13.1), V = 18.6, El. = 111.7, 1-sig unc. =      0.0", b = -52.4, Mot. =   0.150 deg/day, Amor, fading

 2025 YH6     Pos = (08 08, -08.6), V = 18.6, El. = 147.0, 1-sig unc. =      0.3", b = +12.9, Mot. =   1.379 deg/day, 1-opp Apollo, fading

 (163696)     Pos = (09 21, +64.6), V = 18.7, El. = 133.1, 1-sig unc. =      0.0", b = +40.0, Mot. =   0.504 deg/day, Apollo, fading

 2011 YG28    Pos = (10 02, +11.9), V = 18.7, El. = 139.9, 1-sig unc. =      0.0", b = +47.6, Mot. =   1.956 deg/day, Amor, fading

   (1916)     Pos = (01 17, +25.3), V = 18.7, El. =  98.8, 1-sig unc. =      0.0", b = -37.2, Mot. =   0.372 deg/day, Amor, fading

 (153249)     Pos = (12 14, -07.3), V = 18.7, El. = 102.2, 1-sig unc. =      0.0", b = +54.3, Mot. =   0.533 deg/day, Apollo, brightening

 (138877)     Pos = (01 47, +42.0), V = 18.7, El. = 109.5, 1-sig unc. =      0.0", b = -19.7, Mot. =   0.180 deg/day, Apollo, fading

 (248590)     Pos = (02 17, -66.0), V = 18.8, El. =  75.4, 1-sig unc. =      0.0", b = -48.9, Mot. =   0.126 deg/day, Apollo, brightening

  (25916)     Pos = (13 14, +10.7), V = 18.8, El. =  95.3, 1-sig unc. =      0.0", b = +72.8, Mot. =   0.237 deg/day, Amor, brightening

   (1864)     Pos = (14 36, -16.9), V = 18.8, El. =  66.5, 1-sig unc. =      0.0", b = +39.3, Mot. =   0.562 deg/day, Apollo, brightening

 (331509)     Pos = (04 50, +01.0), V = 18.8, El. = 137.9, 1-sig unc. =      0.0", b = -26.2, Mot. =   1.284 deg/day, Amor, fading

  (85804)     Pos = (01 50, +37.9), V = 18.8, El. = 109.2, 1-sig unc. =      0.0", b = -23.6, Mot. =   0.350 deg/day, Amor, fading

 2016 XQ23    Pos = (06 57, +63.3), V = 18.8, El. = 138.7, 1-sig unc. =      0.1", b = +24.7, Mot. =   0.206 deg/day, Amor, brightening

  (90416)     Pos = (05 35, +41.0), V = 18.8, El. = 151.1, 1-sig unc. =      0.0", b = +04.6, Mot. =   0.613 deg/day, PHA, Apollo

 (190161)     Pos = (04 12, +25.8), V = 18.8, El. = 137.2, 1-sig unc. =      0.0", b = -18.4, Mot. =   0.136 deg/day, Amor, fading

 (141079)     Pos = (04 46, -08.7), V = 18.8, El. = 131.4, 1-sig unc. =      0.0", b = -31.7, Mot. =   1.187 deg/day, Apollo, fading

 (789529)     Pos = (04 40, +77.4), V = 18.9, El. = 121.6, 1-sig unc. =      0.1", b = +19.9, Mot. =   1.095 deg/day, Aten, brightening

 (528756)     Pos = (08 37, +19.9), V = 18.9, El. = 161.8, 1-sig unc. =      0.1", b = +31.8, Mot. =   0.189 deg/day, Amor, brightening

 (338172)     Pos = (02 30, -17.3), V = 18.9, El. =  98.8, 1-sig unc. =      0.0", b = -65.3, Mot. =   0.943 deg/day, Amor, fading

 2026 AA      Pos = (03 46, +52.2), V = 18.9, El. = 129.5, 1-sig unc. =      0.4", b = -02.0, Mot. =   4.721 deg/day, 1-opp Apollo, fading

 2025 YK9     Pos = (06 18, -22.7), V = 18.9, El. = 132.7, 1-sig unc. =      0.1", b = -17.1, Mot. =   0.514 deg/day, 1-opp Apollo, brightening

   (4486)     Pos = (07 28, +24.2), V = 18.9, El. = 177.2, 1-sig unc. =      0.0", b = +18.5, Mot. =   0.357 deg/day, PHA, Apollo, fading

 (495930)     Pos = (06 34, +05.5), V = 18.9, El. = 160.1, 1-sig unc. =      0.0", b = -01.3, Mot. =   1.649 deg/day, Apollo, fading

 2002 LX      Pos = (08 05, -00.5), V = 18.9, El. = 154.8, 1-sig unc. =      0.1", b = +16.1, Mot. =   1.122 deg/day, PHA, Apollo, brightening

 2009 AD16    Pos = (11 33, +09.1), V = 18.9, El. = 118.0, 1-sig unc. =      0.0", b = +64.2, Mot. =   2.562 deg/day, Amor, fading

 2015 HC117   Pos = (08 37, +02.1), V = 19.0, El. = 152.5, 1-sig unc. =      0.1", b = +24.3, Mot. =   0.360 deg/day, Amor, brightening

  (34613)     Pos = (09 31, +16.5), V = 19.0, El. = 148.4, 1-sig unc. =      0.0", b = +42.7, Mot. =   0.166 deg/day, Amor, brightening

 2019 TG6     Pos = (06 20, +36.5), V = 19.0, El. = 160.7, 1-sig unc. =      0.0", b = +09.9, Mot. =   0.534 deg/day, Amor

 (138205)     Pos = (01 56, +25.4), V = 19.0, El. = 107.2, 1-sig unc. =      0.0", b = -35.2, Mot. =   0.258 deg/day, Amor, fading

 (612324)     Pos = (06 34, +74.1), V = 19.0, El. = 127.7, 1-sig unc. =      0.3", b = +24.9, Mot. =   0.211 deg/day, Amor, brightening

 2025 SB33    Pos = (03 38, +11.8), V = 19.0, El. = 126.4, 1-sig unc. =      0.1", b = -33.8, Mot. =   1.776 deg/day, 1-opp Amor, fading

 2008 YR2     Pos = (10 01, +25.6), V = 19.0, El. = 143.0, 1-sig unc. =      0.0", b = +52.0, Mot. =   0.362 deg/day, Amor, brightening

 (509352)     Pos = (12 51, -26.1), V = 19.0, El. =  86.3, 1-sig unc. =      0.0", b = +36.8, Mot. =   2.033 deg/day, PHA, Aten, brightening

 (175706)     Pos = (01 22, +11.6), V = 19.1, El. =  95.0, 1-sig unc. =      0.0", b = -50.5, Mot. =   1.275 deg/day, PHA, Apollo, fading

 2006 BC      Pos = (04 48, +65.2), V = 19.1, El. = 130.5, 1-sig unc. =      0.0", b = +12.9, Mot. =   1.865 deg/day, Apollo

 (483566)     Pos = (06 58, +11.7), V = 19.1, El. = 168.4, 1-sig unc. =      0.0", b = +06.8, Mot. =   0.577 deg/day, Amor, fading

 2019 AP6     Pos = (01 17, +51.3), V = 19.1, El. = 106.8, 1-sig unc. =      0.0", b = -11.3, Mot. =   1.756 deg/day, PHA, Aten, fading

 (213053)     Pos = (08 52, +19.6), V = 19.1, El. = 158.4, 1-sig unc. =      0.1", b = +34.9, Mot. =   0.108 deg/day, Amor, brightening

 2026 AE      Pos = (06 08, -06.6), V = 19.1, El. = 146.4, 1-sig unc. =      0.1", b = -12.6, Mot. =   1.775 deg/day, 1-opp Apollo, fading

  (18736)     Pos = (02 17, +17.0), V = 19.1, El. = 109.2, 1-sig unc. =      0.0", b = -41.3, Mot. =   0.096 deg/day, Amor

 (241662)     Pos = (06 51, +13.0), V = 19.2, El. = 168.5, 1-sig unc. =      0.0", b = +05.8, Mot. =   0.601 deg/day, Amor, fading

  (90075)     Pos = (07 31, +09.2), V = 19.2, El. = 166.7, 1-sig unc. =      0.0", b = +12.9, Mot. =   0.319 deg/day, PHA, Apollo

 2018 YJ      Pos = (13 30, +41.4), V = 19.2, El. = 102.5, 1-sig unc. =      0.1", b = +73.6, Mot. =   1.093 deg/day, Amor, brightening

 (159399)     Pos = (13 20, -52.0), V = 19.2, El. =  72.7, 1-sig unc. =      0.0", b = +10.7, Mot. =   0.638 deg/day, Amor, brightening

   (4947)     Pos = (10 28, -31.7), V = 19.2, El. = 109.7, 1-sig unc. =      0.0", b = +22.0, Mot. =   0.285 deg/day, Amor, brightening

   (3122)     Pos = (13 30, -21.9), V = 19.2, El. =  79.6, 1-sig unc. =      0.0", b = +40.1, Mot. =   0.278 deg/day, PHA, Amor, brightening

   (3838)     Pos = (04 21, -00.5), V = 19.2, El. = 131.1, 1-sig unc. =      0.0", b = -33.0, Mot. =   0.220 deg/day, Apollo, fading

 (187026)     Pos = (14 01, -14.0), V = 19.2, El. =  75.3, 1-sig unc. =      0.0", b = +45.5, Mot. =   1.040 deg/day, Aten, brightening

 2025 YP9     Pos = (05 29, +18.2), V = 19.2, El. = 153.9, 1-sig unc. =      0.0", b = -08.8, Mot. =   3.760 deg/day, 1-opp Apollo, fading

 (348776)     Pos = (04 38, +23.9), V = 19.3, El. = 143.0, 1-sig unc. =      0.0", b = -15.2, Mot. =   0.647 deg/day, Amor, fading

 2017 VR12    Pos = (04 57, +69.7), V = 19.3, El. = 128.0, 1-sig unc. =      0.3", b = +16.2, Mot. =   0.349 deg/day, PHA, Amor, brightening

 2025 YW1     Pos = (08 57, +54.5), V = 19.3, El. = 142.8, 1-sig unc. =      0.1", b = +39.9, Mot. =   1.538 deg/day, 1-opp Amor, brightening

 (154656)     Pos = (23 43, +26.6), V = 19.3, El. =  80.2, 1-sig unc. =      0.0", b = -33.8, Mot. =   0.853 deg/day, Apollo, fading

   (8035)     Pos = (02 12, -30.9), V = 19.3, El. =  89.1, 1-sig unc. =      0.0", b = -72.1, Mot. =   0.181 deg/day, Apollo

 2003 AK73    Pos = (03 05, +02.3), V = 19.3, El. = 115.3, 1-sig unc. =      0.1", b = -46.3, Mot. =   2.017 deg/day, Apollo, fading

 (461397)     Pos = (06 41, -50.8), V = 19.3, El. = 106.5, 1-sig unc. =      0.0", b = -22.3, Mot. =   0.442 deg/day, Amor, fading

  (36236)     Pos = (01 27, +14.3), V = 19.3, El. =  97.1, 1-sig unc. =      0.0", b = -47.7, Mot. =   0.443 deg/day, Apollo, fading

   (5587)     Pos = (07 30, -02.5), V = 19.3, El. = 155.2, 1-sig unc. =      0.0", b = +07.5, Mot. =   0.240 deg/day, Amor, brightening

   (2061)     Pos = (08 50, +10.7), V = 19.4, El. = 155.6, 1-sig unc. =      0.0", b = +31.1, Mot. =   0.342 deg/day, Amor, brightening

   (5828)     Pos = (08 37, +13.7), V = 19.4, El. = 159.9, 1-sig unc. =      0.0", b = +29.4, Mot. =   0.396 deg/day, Apollo, brightening

 2025 UY5     Pos = (06 00, +44.3), V = 19.4, El. = 152.4, 1-sig unc. =      0.2", b = +10.2, Mot. =   2.075 deg/day, 1-opp Amor, brightening

   (2329)     Pos = (06 53, -08.7), V = 19.4, El. = 148.4, 1-sig unc. =      0.0", b = -03.6, Mot. =   0.287 deg/day, Apollo, fading

 2022 YU4     Pos = (09 12, +32.5), V = 19.4, El. = 153.0, 1-sig unc. =      0.0", b = +42.7, Mot. =   0.410 deg/day, PHA, Apollo, brightening

 (194268)     Pos = (12 29, -08.6), V = 19.4, El. =  98.3, 1-sig unc. =      0.0", b = +53.9, Mot. =   1.503 deg/day, PHA, Apollo, brightening

 (285625)     Pos = (11 28, +02.2), V = 19.4, El. = 116.6, 1-sig unc. =      0.0", b = +58.0, Mot. =   0.262 deg/day, Amor, brightening

  (90403)     Pos = (13 05, -28.4), V = 19.4, El. =  82.7, 1-sig unc. =      0.0", b = +34.4, Mot. =   1.274 deg/day, PHA, Apollo, brightening

  (23183)     Pos = (14 08, +25.1), V = 19.4, El. =  89.1, 1-sig unc. =      0.0", b = +72.7, Mot. =   0.567 deg/day, Amor

 2012 TA219   Pos = (14 43, +37.0), V = 19.5, El. =  87.9, 1-sig unc. =      0.0", b = +64.5, Mot. =   0.643 deg/day, Amor

 (152679)     Pos = (01 38, +06.0), V = 19.5, El. =  96.4, 1-sig unc. =      0.0", b = -55.0, Mot. =   0.609 deg/day, Amor, fading

 (138847)     Pos = (05 07, -15.7), V = 19.5, El. = 130.1, 1-sig unc. =      0.0", b = -30.1, Mot. =   0.279 deg/day, Amor, fading

 2004 XL29    Pos = (03 13, -42.6), V = 19.5, El. =  94.0, 1-sig unc. =      0.1", b = -57.4, Mot. =   1.618 deg/day, Apollo, fading

 (526742)     Pos = (04 23, -05.5), V = 19.5, El. = 128.6, 1-sig unc. =      0.0", b = -35.4, Mot. =   1.121 deg/day, Aten, fading

 (531899)     Pos = (02 31, +82.6), V = 19.5, El. = 114.3, 1-sig unc. =      0.1", b = +20.4, Mot. =   1.330 deg/day, PHA, Aten, brightening

 2025 XN1     Pos = (06 09, -44.6), V = 19.5, El. = 111.3, 1-sig unc. =      0.3", b = -25.9, Mot. =   4.898 deg/day, 1-opp Apollo, fading

  (86039)     Pos = (14 36, -21.1), V = 19.5, El. =  65.2, 1-sig unc. =      0.0", b = +35.5, Mot. =   0.423 deg/day, PHA, Apollo, fading

 2020 EN      Pos = (23 53, -09.3), V = 19.5, El. =  66.6, 1-sig unc. =      0.2", b = -67.4, Mot. =   1.353 deg/day, PHA, Apollo, brightening

  (89959)     Pos = (14 50, +15.1), V = 19.5, El. =  75.7, 1-sig unc. =      0.0", b = +59.9, Mot. =   0.699 deg/day, PHA, Apollo, fading

 2025 YJ6     Pos = (08 49, +32.9), V = 19.5, El. = 157.5, 1-sig unc. =      0.3", b = +37.9, Mot. =   1.530 deg/day, 1-opp Apollo, fading

 2016 NN39    Pos = (09 55, -04.9), V = 19.5, El. = 133.4, 1-sig unc. =      0.2", b = +36.6, Mot. =   0.625 deg/day, Amor, brightening

 (614313)     Pos = (09 20, +25.7), V = 19.5, El. = 152.3, 1-sig unc. =      0.1", b = +43.0, Mot. =   0.342 deg/day, Apollo, brightening

 (329571)     Pos = (08 13, -68.4), V = 19.5, El. =  88.9, 1-sig unc. =      0.0", b = -18.0, Mot. =   0.518 deg/day, Amor, fading

 (112985)     Pos = (01 44, -14.8), V = 19.5, El. =  90.0, 1-sig unc. =      0.0", b = -72.6, Mot. =   0.330 deg/day, Amor, fading

   (5646)     Pos = (09 47, +24.9), V = 19.5, El. = 146.1, 1-sig unc. =      0.0", b = +48.8, Mot. =   0.235 deg/day, Amor, brightening

   (6569)     Pos = (11 44, +21.5), V = 19.5, El. = 119.1, 1-sig unc. =      0.0", b = +73.7, Mot. =   0.368 deg/day, Amor, brightening

 (162998)     Pos = (00 33, +05.2), V = 19.6, El. =  81.3, 1-sig unc. =      0.0", b = -57.3, Mot. =   0.678 deg/day, PHA, Amor

 (219071)     Pos = (00 44, -06.8), V = 19.6, El. =  79.1, 1-sig unc. =      0.0", b = -69.6, Mot. =   0.530 deg/day, Apollo, fading

 (137158)     Pos = (15 02, -01.4), V = 19.6, El. =  65.7, 1-sig unc. =      0.0", b = +47.4, Mot. =   0.131 deg/day, Apollo, fading

 (228587)     Pos = (09 41, +39.3), V = 19.6, El. = 145.4, 1-sig unc. =      0.0", b = +48.9, Mot. =   0.939 deg/day, Amor

 (111253)     Pos = (11 51, -33.2), V = 19.6, El. =  94.8, 1-sig unc. =      0.0", b = +28.1, Mot. =   0.430 deg/day, PHA, Apollo, brightening

   (3691)     Pos = (00 14, +19.3), V = 19.6, El. =  83.0, 1-sig unc. =      0.0", b = -42.7, Mot. =   0.278 deg/day, Amor, fading

 (162038)     Pos = (10 26, +06.9), V = 19.6, El. = 132.6, 1-sig unc. =      0.0", b = +50.0, Mot. =   0.198 deg/day, Amor, brightening

 2015 NU13    Pos = (12 02, -03.4), V = 19.6, El. = 106.5, 1-sig unc. =      0.0", b = +57.2, Mot. =   0.530 deg/day, PHA, Apollo, brightening

 (455550)     Pos = (05 35, -06.4), V = 19.6, El. = 141.7, 1-sig unc. =      0.0", b = -19.9, Mot. =   0.225 deg/day, Apollo, fading

 (345722)     Pos = (14 29, +20.1), V = 19.7, El. =  82.6, 1-sig unc. =      0.0", b = +66.6, Mot. =   0.888 deg/day, Aten, brightening

 2025 RD5     Pos = (14 03, +04.2), V = 19.7, El. =  81.6, 1-sig unc. =      0.1", b = +61.4, Mot. =   1.733 deg/day, 1-opp PHA, Apollo, fading

 2020 BQ3     Pos = (09 43, +50.9), V = 19.7, El. = 140.0, 1-sig unc. =      0.0", b = +47.5, Mot. =   0.570 deg/day, Amor, fading

 2012 XF55    Pos = (03 39, +10.7), V = 19.7, El. = 126.2, 1-sig unc. =      0.0", b = -34.4, Mot. =   0.713 deg/day, Apollo, fading

 (849935)     Pos = (03 01, +22.6), V = 19.7, El. = 120.8, 1-sig unc. =      0.0", b = -31.1, Mot. =   0.787 deg/day, Amor, fading

 2015 VL105   Pos = (08 36, -33.7), V = 19.7, El. = 121.2, 1-sig unc. =      0.0", b = +04.3, Mot. =   0.545 deg/day, Amor, fading

 (230599)     Pos = (02 24, -17.2), V = 19.7, El. =  97.6, 1-sig unc. =      0.0", b = -66.5, Mot. =   1.218 deg/day, Apollo, fading

 2025 XC7     Pos = (10 25, +34.5), V = 19.7, El. = 137.7, 1-sig unc. =      0.1", b = +58.0, Mot. =   3.932 deg/day, 1-opp Amor, fading

 (859506)     Pos = (10 38, +17.1), V = 19.7, El. = 133.2, 1-sig unc. =      0.0", b = +57.7, Mot. =   0.385 deg/day, Amor

 2025 OZ12    Pos = (07 00, -00.7), V = 19.7, El. = 156.7, 1-sig unc. =      0.1", b = +01.7, Mot. =   0.644 deg/day, 1-opp Apollo, fading

 (349925)     Pos = (03 01, -00.3), V = 19.7, El. = 113.3, 1-sig unc. =      0.0", b = -48.9, Mot. =   0.232 deg/day, Apollo

 (860019)     Pos = (11 22, -19.9), V = 19.7, El. = 107.3, 1-sig unc. =      0.0", b = +38.2, Mot. =   1.261 deg/day, Amor, brightening

 (827574)     Pos = (09 27, +05.0), V = 19.7, El. = 144.8, 1-sig unc. =      0.0", b = +36.6, Mot. =   0.300 deg/day, Amor, brightening

 (162058)     Pos = (08 02, +22.5), V = 19.8, El. = 170.3, 1-sig unc. =      0.0", b = +25.0, Mot. =   0.392 deg/day, Amor, fading

 (816502)     Pos = (06 50, +39.3), V = 19.8, El. = 161.7, 1-sig unc. =      0.0", b = +16.5, Mot. =   1.103 deg/day, Amor, fading

 (307070)     Pos = (09 58, +10.2), V = 19.8, El. = 140.3, 1-sig unc. =      0.0", b = +45.9, Mot. =   0.894 deg/day, Apollo, brightening

 (509935)     Pos = (23 50, +01.6), V = 19.8, El. =  70.1, 1-sig unc. =      0.0", b = -57.7, Mot. =   1.827 deg/day, PHA, Apollo, brightening

 2025 TD12    Pos = (06 53, +15.9), V = 19.8, El. = 171.1, 1-sig unc. =      0.2", b = +07.5, Mot. =   0.791 deg/day, 1-opp Amor, fading

 2025 XT1     Pos = (08 35, -06.7), V = 19.8, El. = 145.8, 1-sig unc. =      2.0", b = +19.3, Mot. =   0.441 deg/day, 1-opp Amor, brightening

 (612227)     Pos = (23 49, -07.3), V = 19.8, El. =  66.3, 1-sig unc. =      0.0", b = -65.3, Mot. =   0.876 deg/day, PHA, Apollo, fading

 (348400)     Pos = (04 16, +06.9), V = 19.8, El. = 133.2, 1-sig unc. =      0.0", b = -30.0, Mot. =   0.227 deg/day, PHA, Amor, fading

 (154555)     Pos = (13 40, +54.7), V = 19.8, El. = 105.2, 1-sig unc. =      0.0", b = +61.1, Mot. =   0.422 deg/day, Apollo, brightening

  (53319)     Pos = (14 38, -03.8), V = 19.8, El. =  70.3, 1-sig unc. =      0.0", b = +49.7, Mot. =   0.528 deg/day, PHA, Apollo, brightening

  (54690)     Pos = (15 12, +25.4), V = 19.8, El. =  76.6, 1-sig unc. =      0.1", b = +58.6, Mot. =   0.579 deg/day, Amor

 2015 NV28    Pos = (09 59, -42.1), V = 19.8, El. = 105.9, 1-sig unc. =      0.4", b = +10.2, Mot. =   0.634 deg/day, Amor, brightening

   (3552)     Pos = (12 22, -04.7), V = 19.8, El. = 101.5, 1-sig unc. =      0.0", b = +57.4, Mot. =   0.074 deg/day, Amor, brightening

 (329614)     Pos = (00 35, -01.7), V = 19.8, El. =  79.0, 1-sig unc. =      0.1", b = -64.3, Mot. =   1.373 deg/day, PHA, Apollo, fading

 2016 YV8     Pos = (10 19, +27.3), V = 19.8, El. = 139.2, 1-sig unc. =      0.0", b = +56.2, Mot. =   1.292 deg/day, Amor, fading

 2017 SL14    Pos = (10 18, -17.5), V = 19.8, El. = 121.1, 1-sig unc. =      0.1", b = +31.8, Mot. =   1.081 deg/day, Amor

 (417167)     Pos = (02 49, +04.0), V = 19.8, El. = 112.2, 1-sig unc. =      0.0", b = -47.8, Mot. =   1.166 deg/day, Amor, fading

 2025 SP27    Pos = (09 09, -10.5), V = 19.8, El. = 137.8, 1-sig unc. =      0.2", b = +24.2, Mot. =   0.436 deg/day, 1-opp Amor, fading

 2025 YC10    Pos = (07 49, +02.6), V = 19.9, El. = 159.1, 1-sig unc. =      0.2", b = +14.0, Mot. =   0.779 deg/day, 1-opp Amor, brightening

 (177651)     Pos = (02 37, -09.0), V = 19.9, El. = 104.2, 1-sig unc. =      0.0", b = -59.1, Mot. =   0.721 deg/day, Apollo, fading

   (1981)     Pos = (23 54, +39.2), V = 19.9, El. =  88.7, 1-sig unc. =      0.0", b = -22.3, Mot. =   0.256 deg/day, PHA, Apollo, fading

 (137052)     Pos = (06 06, +38.5), V = 19.9, El. = 157.4, 1-sig unc. =      0.0", b = +08.5, Mot. =   0.543 deg/day, Apollo, fading

 2011 YU62    Pos = (04 18, -15.3), V = 19.9, El. = 121.7, 1-sig unc. =      0.0", b = -40.9, Mot. =   0.319 deg/day, Apollo

 (312070)     Pos = (08 49, -72.5), V = 19.9, El. =  84.1, 1-sig unc. =      0.1", b = -17.7, Mot. =   1.425 deg/day, PHA, Aten, fading

  (54401)     Pos = (05 56, +01.9), V = 19.9, El. = 151.4, 1-sig unc. =      0.0", b = -11.3, Mot. =   0.317 deg/day, Amor, fading

 2025 YF1     Pos = (05 53, +14.4), V = 19.9, El. = 158.0, 1-sig unc. =      0.6", b = -06.1, Mot. =   2.195 deg/day, 1-opp Apollo, fading

 (162142)     Pos = (23 37, +24.3), V = 19.9, El. =  78.0, 1-sig unc. =      0.0", b = -35.5, Mot. =   0.392 deg/day, Aten, fading

 2025 YH5     Pos = (14 39, +21.1), V = 19.9, El. =  80.9, 1-sig unc. =      1.9", b = +64.7, Mot. =   2.117 deg/day, 1-opp PHA, Apollo, brightening

 (138971)     Pos = (02 19, +31.9), V = 19.9, El. = 113.6, 1-sig unc. =      0.0", b = -27.4, Mot. =   0.264 deg/day, PHA, Apollo, fading

  (16834)     Pos = (00 10, +13.8), V = 20.0, El. =  79.6, 1-sig unc. =      0.0", b = -47.9, Mot. =   0.384 deg/day, Apollo, fading

 2018 YU      Pos = (08 15, -15.8), V = 20.0, El. = 139.6, 1-sig unc. =      0.1", b = +10.5, Mot. =   0.651 deg/day, Apollo, brightening

 (455213)     Pos = (01 04, +17.6), V = 20.0, El. =  93.1, 1-sig unc. =      0.0", b = -45.2, Mot. =   0.965 deg/day, Amor, fading

 2017 WX14    Pos = (14 15, -22.9), V = 20.0, El. =  69.4, 1-sig unc. =      0.4", b = +36.0, Mot. =   1.364 deg/day, Amor, fading

 (523799)     Pos = (08 37, -31.8), V = 20.0, El. = 122.9, 1-sig unc. =      0.0", b = +05.5, Mot. =   0.755 deg/day, Amor, brightening

 2025 VD24    Pos = (11 22, -06.0), V = 20.0, El. = 114.4, 1-sig unc. =      0.0", b = +50.4, Mot. =   0.816 deg/day, 1-opp Amor, brightening

 (311554)     Pos = (01 10, -38.0), V = 20.0, El. =  74.7, 1-sig unc. =      0.1", b = -78.4, Mot. =   1.372 deg/day, Aten, fading

 (294739)     Pos = (12 11, -44.9), V = 20.0, El. =  85.8, 1-sig unc. =      0.0", b = +17.4, Mot. =   0.557 deg/day, PHA, Apollo

  (18882)     Pos = (23 27, +37.8), V = 20.0, El. =  83.5, 1-sig unc. =      0.0", b = -22.1, Mot. =   0.482 deg/day, Amor, fading

 (241596)     Pos = (07 33, -15.1), V = 20.0, El. = 142.6, 1-sig unc. =      0.0", b = +02.0, Mot. =   0.403 deg/day, Amor, fading

 2017 UW42    Pos = (12 56, -13.3), V = 20.0, El. =  90.4, 1-sig unc. =      0.0", b = +49.6, Mot. =   0.668 deg/day, Amor, brightening

 (504256)     Pos = (00 17, +23.8), V = 20.0, El. =  85.7, 1-sig unc. =      0.0", b = -38.4, Mot. =   0.661 deg/day, PHA, Amor, brightening

 2015 XJ261   Pos = (09 20, +09.9), V = 20.0, El. = 148.7, 1-sig unc. =      0.0", b = +37.4, Mot. =   0.561 deg/day, Amor, brightening

 (138175)     Pos = (03 58, +39.8), V = 20.0, El. = 134.0, 1-sig unc. =      0.0", b = -10.2, Mot. =   0.288 deg/day, PHA, Apollo, fading

  (98943)     Pos = (01 47, +11.7), V = 20.0, El. = 100.6, 1-sig unc. =      0.0", b = -48.9, Mot. =   0.484 deg/day, Apollo, fading

 2025 YX7     Pos = (06 34, -32.0), V = 20.0, El. = 124.7, 1-sig unc. =      0.3", b = -17.4, Mot. =   2.388 deg/day, 1-opp Apollo, fading

 (387793)     Pos = (00 49, +22.8), V = 20.0, El. =  92.0, 1-sig unc. =      0.0", b = -40.1, Mot. =   0.615 deg/day, Apollo, fading

 2025 YF3     Pos = (08 56, -12.8), V = 20.0, El. = 137.8, 1-sig unc. =      0.2", b = +20.3, Mot. =   4.136 deg/day, 1-opp Amor, fading

 (162215)     Pos = (00 40, -44.7), V = 20.0, El. =  67.7, 1-sig unc. =      0.1", b = -72.3, Mot. =   0.580 deg/day, Apollo, fading

 2025 XF5     Pos = (09 51, +60.2), V = 20.0, El. = 133.7, 1-sig unc. =      0.2", b = +45.1, Mot. =   4.416 deg/day, 1-opp Amor, fading

  (96315)     Pos = (04 27, +26.8), V = 20.0, El. = 140.6, 1-sig unc. =      0.0", b = -15.3, Mot. =   0.246 deg/day, Apollo, fading

 (446935)     Pos = (02 04, +17.0), V = 20.0, El. = 106.3, 1-sig unc. =      0.0", b = -42.5, Mot. =   0.510 deg/day, Amor, fading

  (17182)     Pos = (00 08, +12.0), V = 20.1, El. =  78.6, 1-sig unc. =      0.0", b = -49.5, Mot. =   0.316 deg/day, Apollo

   (5863)     Pos = (12 42, -06.0), V = 20.1, El. =  96.5, 1-sig unc. =      0.0", b = +56.8, Mot. =   0.300 deg/day, Amor, brightening

  (35396)     Pos = (09 57, +05.4), V = 20.1, El. = 138.4, 1-sig unc. =      0.0", b = +43.2, Mot. =   0.238 deg/day, PHA, Apollo, brightening

 1999 SF10    Pos = (09 27, +32.3), V = 20.1, El. = 150.1, 1-sig unc. =      0.0", b = +45.6, Mot. =   0.741 deg/day, Apollo, fading

 (308127)     Pos = (00 49, +46.2), V = 20.1, El. = 100.7, 1-sig unc. =      0.0", b = -16.7, Mot. =   0.482 deg/day, Amor

 2009 KQ4     Pos = (08 06, +37.9), V = 20.1, El. = 161.3, 1-sig unc. =      0.1", b = +30.4, Mot. =   0.422 deg/day, Amor, brightening

 2025 OT45    Pos = (02 22, -09.1), V = 20.1, El. = 100.7, 1-sig unc. =      0.0", b = -62.0, Mot. =   0.927 deg/day, 1-opp Amor, fading

 (348314)     Pos = (12 05, -30.8), V = 20.1, El. =  93.5, 1-sig unc. =      0.0", b = +31.0, Mot. =   0.700 deg/day, PHA, Apollo, brightening

 (422638)     Pos = (07 45, +53.3), V = 20.1, El. = 148.5, 1-sig unc. =      0.0", b = +29.3, Mot. =   0.905 deg/day, Apollo, brightening

 (418198)     Pos = (03 02, -36.9), V = 20.1, El. =  95.3, 1-sig unc. =      0.0", b = -60.9, Mot. =   0.780 deg/day, Aten, fading

 2014 YR15    Pos = (01 33, +33.0), V = 20.1, El. = 104.6, 1-sig unc. =      0.0", b = -29.1, Mot. =   0.823 deg/day, PHA, Aten, fading

 (525319)     Pos = (01 22, +47.3), V = 20.1, El. = 106.5, 1-sig unc. =      0.0", b = -15.3, Mot. =   0.986 deg/day, Amor

   (1580)     Pos = (00 44, -38.4), V = 20.1, El. =  69.7, 1-sig unc. =      0.0", b = -78.6, Mot. =   0.403 deg/day, Amor, fading

 (467609)     Pos = (04 26, +41.8), V = 20.1, El. = 138.8, 1-sig unc. =      0.0", b = -05.2, Mot. =   0.566 deg/day, Apollo, fading

 2026 AF      Pos = (07 52, +43.2), V = 20.1, El. = 157.9, 1-sig unc. =      0.4", b = +28.8, Mot. =   1.940 deg/day, 1-opp Apollo, fading

  (23714)     Pos = (11 19, +08.0), V = 20.1, El. = 120.8, 1-sig unc. =      0.0", b = +61.0, Mot. =   0.184 deg/day, Amor, brightening

 (442609)     Pos = (04 25, -05.4), V = 20.1, El. = 129.2, 1-sig unc. =      0.0", b = -34.8, Mot. =   0.410 deg/day, Amor, fading

 2003 HP32    Pos = (06 11, +15.2), V = 20.2, El. = 162.4, 1-sig unc. =      0.0", b = -01.8, Mot. =   0.389 deg/day, Apollo, fading

 2018 YE2     Pos = (14 01, +34.0), V = 20.2, El. =  94.1, 1-sig unc. =      0.1", b = +73.5, Mot. =   1.225 deg/day, Amor, fading

  (17188)     Pos = (02 37, +31.2), V = 20.2, El. = 117.4, 1-sig unc. =      0.0", b = -26.4, Mot. =   0.283 deg/day, Apollo, fading

 (424965)     Pos = (12 33, -31.7), V = 20.2, El. =  87.7, 1-sig unc. =      0.0", b = +31.0, Mot. =   0.698 deg/day, Aten, brightening

 (242216)     Pos = (07 49, -12.4), V = 20.2, El. = 144.7, 1-sig unc. =      0.0", b = +06.8, Mot. =   0.344 deg/day, PHA, Amor, brightening

 2025 XY6     Pos = (06 17, +01.1), V = 20.2, El. = 154.1, 1-sig unc. =      0.1", b = -07.0, Mot. =   3.767 deg/day, 1-opp Amor, fading

 (162900)     Pos = (14 51, -14.4), V = 20.2, El. =  63.8, 1-sig unc. =      0.0", b = +39.4, Mot. =   0.422 deg/day, Amor

 2023 UN9     Pos = (01 42, +63.3), V = 20.2, El. = 112.2, 1-sig unc. =      0.0", b = +01.0, Mot. =   0.340 deg/day, Aten, fading

 (155110)     Pos = (02 53, -08.0), V = 20.2, El. = 108.2, 1-sig unc. =      0.0", b = -55.4, Mot. =   0.107 deg/day, Apollo, fading

 2011 MV      Pos = (02 25, +69.4), V = 20.2, El. = 116.4, 1-sig unc. =      0.0", b = +08.1, Mot. =   1.939 deg/day, Amor, fading

 2013 AE69    Pos = (04 21, +69.9), V = 20.2, El. = 125.5, 1-sig unc. =      0.1", b = +14.0, Mot. =   0.356 deg/day, Amor

 (330825)     Pos = (09 02, +19.4), V = 20.2, El. = 156.0, 1-sig unc. =      0.0", b = +37.1, Mot. =   0.280 deg/day, Amor, brightening

 2011 YX10    Pos = (05 44, -01.5), V = 20.2, El. = 146.7, 1-sig unc. =      0.1", b = -15.7, Mot. =   1.675 deg/day, Aten, fading

 (159608)     Pos = (10 39, +58.7), V = 20.2, El. = 129.2, 1-sig unc. =      0.0", b = +51.0, Mot. =   0.580 deg/day, Amor

 (341816)     Pos = (12 05, +11.1), V = 20.2, El. = 111.3, 1-sig unc. =      0.0", b = +70.5, Mot. =   0.463 deg/day, Amor, brightening

 (313809)     Pos = (03 34, -26.6), V = 20.2, El. = 106.9, 1-sig unc. =      0.1", b = -53.9, Mot. =   1.456 deg/day, Apollo, fading

 2017 XO60    Pos = (06 01, +41.7), V = 20.3, El. = 154.4, 1-sig unc. =      0.0", b = +09.0, Mot. =   0.607 deg/day, Apollo, fading

 2020 WC5     Pos = (03 10, +12.4), V = 20.3, El. = 120.0, 1-sig unc. =      0.1", b = -38.1, Mot. =   1.395 deg/day, Amor, fading

 (511684)     Pos = (06 50, +19.8), V = 20.3, El. = 172.6, 1-sig unc. =      0.0", b = +08.5, Mot. =   1.006 deg/day, PHA, Apollo, fading

 (613763)     Pos = (06 37, -05.5), V = 20.3, El. = 150.4, 1-sig unc. =      0.1", b = -05.8, Mot. =   0.694 deg/day, Amor, fading

 (350964)     Pos = (05 13, -50.2), V = 20.3, El. = 102.4, 1-sig unc. =      0.1", b = -36.0, Mot. =   1.089 deg/day, Amor, brightening

 2025 RV1     Pos = (09 56, +37.5), V = 20.3, El. = 143.1, 1-sig unc. =      0.1", b = +51.9, Mot. =   0.328 deg/day, 1-opp Amor, fading

 2024 AN2     Pos = (03 53, -17.9), V = 20.3, El. = 115.5, 1-sig unc. =      0.0", b = -47.3, Mot. =   0.544 deg/day, PHA, Apollo, brightening

 2025 XY5     Pos = (06 28, -12.9), V = 20.3, El. = 142.8, 1-sig unc. =      2.1", b = -10.9, Mot. =   2.184 deg/day, 1-opp Aten, fading

 2019 AS4     Pos = (05 17, +29.4), V = 20.3, El. = 151.7, 1-sig unc. =      0.1", b = -04.9, Mot. =   1.212 deg/day, Amor, fading

 (138883)     Pos = (11 41, -15.5), V = 20.3, El. = 105.8, 1-sig unc. =      0.0", b = +44.1, Mot. =   0.187 deg/day, Amor, brightening

  (54660)     Pos = (23 34, -05.6), V = 20.3, El. =  63.5, 1-sig unc. =      0.0", b = -61.7, Mot. =   1.114 deg/day, Amor, fading

 (484403)     Pos = (05 17, +04.0), V = 20.3, El. = 145.3, 1-sig unc. =      0.1", b = -18.8, Mot. =   1.319 deg/day, Amor, fading

 2015 BJ514   Pos = (13 08, -28.4), V = 20.3, El. =  82.0, 1-sig unc. =      0.1", b = +34.3, Mot. =   1.229 deg/day, Apollo, brightening

 (818298)     Pos = (07 42, -48.0), V = 20.3, El. = 109.6, 1-sig unc. =      0.0", b = -12.0, Mot. =   0.652 deg/day, Amor

 2025 XA4     Pos = (11 12, -08.3), V = 20.3, El. = 115.5, 1-sig unc. =      1.2", b = +47.1, Mot. =   2.901 deg/day, 1-opp Amor, fading

 (483472)     Pos = (05 44, +29.1), V = 20.3, El. = 157.3, 1-sig unc. =      0.0", b = -00.3, Mot. =   0.416 deg/day, Apollo, fading

 (218863)     Pos = (23 29, -08.9), V = 20.3, El. =  61.2, 1-sig unc. =      0.0", b = -63.4, Mot. =   0.438 deg/day, Apollo, brightening

 2006 BA9     Pos = (02 30, +45.3), V = 20.3, El. = 117.8, 1-sig unc. =      0.2", b = -14.1, Mot. =   2.801 deg/day, Aten, fading

 (830529)     Pos = (06 30, +32.1), V = 20.3, El. = 165.2, 1-sig unc. =      0.0", b = +09.9, Mot. =   0.293 deg/day, Amor, fading

  (40263)     Pos = (00 58, -23.8), V = 20.4, El. =  76.6, 1-sig unc. =      0.0", b = -86.3, Mot. =   0.730 deg/day, Amor, fading

 (433953)     Pos = (09 47, -08.2), V = 20.4, El. = 132.7, 1-sig unc. =      0.0", b = +33.2, Mot. =   0.506 deg/day, PHA, Apollo, brightening

 2018 VL7     Pos = (03 46, +49.4), V = 20.4, El. = 130.3, 1-sig unc. =      0.1", b = -04.1, Mot. =   0.873 deg/day, Apollo, fading

 (752296)     Pos = (12 27, +18.2), V = 20.4, El. = 108.5, 1-sig unc. =      0.0", b = +79.5, Mot. =   1.520 deg/day, Apollo

 (462736)     Pos = (14 54, +59.6), V = 20.4, El. =  97.9, 1-sig unc. =      0.1", b = +51.3, Mot. =   0.591 deg/day, Apollo, brightening

 2012 BC124   Pos = (11 37, +45.3), V = 20.4, El. = 123.4, 1-sig unc. =      0.0", b = +66.5, Mot. =   0.690 deg/day, Amor, brightening

 2025 YA6     Pos = (07 55, +07.8), V = 20.4, El. = 163.3, 1-sig unc. =      0.4", b = +17.5, Mot. =   0.449 deg/day, 1-opp Amor, brightening

 2010 AK2     Pos = (11 49, -46.0), V = 20.4, El. =  88.6, 1-sig unc. =      0.5", b = +15.5, Mot. =   1.962 deg/day, Apollo

 (755140)     Pos = (04 34, +34.9), V = 20.4, El. = 141.8, 1-sig unc. =      0.0", b = -08.7, Mot. =   0.824 deg/day, Apollo, fading

 (163014)     Pos = (14 16, -01.9), V = 20.4, El. =  76.2, 1-sig unc. =      0.0", b = +54.5, Mot. =   0.515 deg/day, PHA, Apollo

 (415027)     Pos = (05 53, -21.0), V = 20.4, El. = 131.9, 1-sig unc. =      0.0", b = -21.9, Mot. =   0.477 deg/day, Apollo, fading

 (162741)     Pos = (11 02, +26.8), V = 20.4, El. = 129.5, 1-sig unc. =      0.0", b = +65.7, Mot. =   0.210 deg/day, Amor, brightening

   (6456)     Pos = (04 54, +31.4), V = 20.4, El. = 146.5, 1-sig unc. =      0.0", b = -07.7, Mot. =   0.193 deg/day, Amor, fading

 (455193)     Pos = (10 56, +33.2), V = 20.4, El. = 131.4, 1-sig unc. =      0.0", b = +64.4, Mot. =   0.483 deg/day, Amor, brightening

  (22771)     Pos = (03 48, +37.8), V = 20.4, El. = 132.3, 1-sig unc. =      0.0", b = -13.0, Mot. =   0.199 deg/day, Apollo, fading

   (3271)     Pos = (05 21, -13.3), V = 20.4, El. = 134.1, 1-sig unc. =      0.0", b = -26.0, Mot. =   0.285 deg/day, Amor, fading

 2007 AV2     Pos = (09 55, +01.6), V = 20.4, El. = 137.1, 1-sig unc. =      8.6", b = +40.6, Mot. =   1.057 deg/day, PHA, Apollo, brightening

 (162421)     Pos = (14 53, -25.4), V = 20.4, El. =  60.4, 1-sig unc. =      0.0", b = +29.8, Mot. =   0.952 deg/day, PHA, Aten

 2018 AX      Pos = (07 42, -11.2), V = 20.4, El. = 146.1, 1-sig unc. =      0.0", b = +06.0, Mot. =   1.494 deg/day, Amor, fading

 (679885)     Pos = (13 00, -05.7), V = 20.4, El. =  92.5, 1-sig unc. =      0.1", b = +57.1, Mot. =   0.781 deg/day, PHA, Apollo, brightening

 (265661)     Pos = (08 28, -05.0), V = 20.4, El. = 148.1, 1-sig unc. =      0.0", b = +18.8, Mot. =   0.405 deg/day, Apollo, brightening

 2018 EM15    Pos = (09 47, +23.7), V = 20.4, El. = 146.1, 1-sig unc. =      0.2", b = +48.4, Mot. =   0.170 deg/day, Amor, brightening

 2025 YA2     Pos = (10 27, +11.4), V = 20.5, El. = 134.1, 1-sig unc. =      1.7", b = +52.6, Mot. =   1.799 deg/day, 1-opp Amor, brightening

 (153271)     Pos = (09 23, +54.0), V = 20.5, El. = 140.6, 1-sig unc. =      0.0", b = +43.6, Mot. =   0.346 deg/day, Apollo, brightening

 (370702)     Pos = (13 11, -12.2), V = 20.5, El. =  87.3, 1-sig unc. =      0.1", b = +50.3, Mot. =   0.781 deg/day, Amor, brightening

 (839556)     Pos = (09 25, -55.6), V = 20.5, El. =  97.8, 1-sig unc. =      0.0", b = -03.7, Mot. =   0.328 deg/day, Amor, brightening

 2006 WK3     Pos = (07 06, +43.0), V = 20.5, El. = 159.0, 1-sig unc. =      0.0", b = +20.6, Mot. =   0.502 deg/day, Amor, fading

 2025 YN4     Pos = (05 53, +20.7), V = 20.5, El. = 159.8, 1-sig unc. =      0.3", b = -02.8, Mot. =   0.982 deg/day, 1-opp PHA, Apollo

 (357439)     Pos = (00 18, -33.6), V = 20.5, El. =  65.5, 1-sig unc. =      0.1", b = -80.3, Mot. =   1.092 deg/day, PHA, Apollo, brightening

 (138911)     Pos = (03 30, +14.9), V = 20.5, El. = 125.4, 1-sig unc. =      0.0", b = -33.0, Mot. =   0.097 deg/day, Amor, fading

 2025 YK5     Pos = (06 47, +25.8), V = 20.5, El. = 171.6, 1-sig unc. =      0.8", b = +10.5, Mot. =   2.884 deg/day, 1-opp Amor, fading

 2020 YQ4     Pos = (11 40, +34.9), V = 20.5, El. = 122.4, 1-sig unc. =      0.0", b = +72.9, Mot. =   1.043 deg/day, Amor

 (138925)     Pos = (03 34, -66.1), V = 20.5, El. =  82.1, 1-sig unc. =      0.0", b = -43.6, Mot. =   0.465 deg/day, Amor

 2017 NQ6     Pos = (14 53, +04.8), V = 20.5, El. =  70.4, 1-sig unc. =      0.1", b = +53.3, Mot. =   1.584 deg/day, Apollo, fading

 (217807)     Pos = (11 59, +22.6), V = 20.5, El. = 116.1, 1-sig unc. =      0.0", b = +77.2, Mot. =   0.155 deg/day, Amor, brightening

 (163000)     Pos = (12 08, +06.0), V = 20.5, El. = 108.8, 1-sig unc. =      0.0", b = +66.5, Mot. =   0.461 deg/day, Amor, brightening

  (13651)     Pos = (03 17, -01.1), V = 20.5, El. = 116.6, 1-sig unc. =      0.0", b = -46.4, Mot. =   0.305 deg/day, PHA, Apollo, fading

 (523603)     Pos = (14 24, +03.6), V = 20.5, El. =  76.5, 1-sig unc. =      0.0", b = +57.7, Mot. =   0.619 deg/day, Apollo

 (252558)     Pos = (05 22, +32.5), V = 20.5, El. = 152.0, 1-sig unc. =      0.0", b = -02.3, Mot. =   0.685 deg/day, Apollo, fading

 (507847)     Pos = (14 38, -09.5), V = 20.5, El. =  68.4, 1-sig unc. =      0.0", b = +45.2, Mot. =   0.624 deg/day, Amor, fading

 2025 XS1     Pos = (00 45, +06.8), V = 20.5, El. =  84.6, 1-sig unc. =      0.3", b = -56.0, Mot. =   4.263 deg/day, 1-opp PHA, Apollo, fading

 (441641)     Pos = (03 01, +25.8), V = 20.5, El. = 121.5, 1-sig unc. =      0.0", b = -28.5, Mot. =   0.215 deg/day, PHA, Apollo, fading

   (1917)     Pos = (14 25, -09.2), V = 20.5, El. =  71.6, 1-sig unc. =      0.0", b = +47.3, Mot. =   0.204 deg/day, Amor, brightening

 (189058)     Pos = (07 09, -00.4), V = 20.5, El. = 157.3, 1-sig unc. =      0.0", b = +03.8, Mot. =   0.322 deg/day, Amor, fading

 (333521)     Pos = (01 41, +30.6), V = 20.5, El. = 105.4, 1-sig unc. =      0.0", b = -31.0, Mot. =   0.536 deg/day, PHA, Apollo, fading

 (204131)     Pos = (00 59, +13.3), V = 20.5, El. =  90.3, 1-sig unc. =      0.1", b = -49.5, Mot. =   2.106 deg/day, Apollo, fading

 (459386)     Pos = (11 27, +28.3), V = 20.5, El. = 124.4, 1-sig unc. =      0.0", b = +71.2, Mot. =   0.120 deg/day, Apollo, brightening

 (416851)     Pos = (03 37, +06.1), V = 20.5, El. = 124.1, 1-sig unc. =      0.0", b = -37.9, Mot. =   0.635 deg/day, Amor, fading

 2019 GU2     Pos = (00 45, +54.7), V = 20.6, El. = 103.2, 1-sig unc. =      0.2", b = -08.2, Mot. =   0.205 deg/day, Apollo, brightening

 (174881)     Pos = (05 36, -34.2), V = 20.6, El. = 118.5, 1-sig unc. =      0.0", b = -29.6, Mot. =   0.897 deg/day, Apollo, fading

 (870028)     Pos = (13 23, +00.9), V = 20.6, El. =  89.5, 1-sig unc. =      0.0", b = +62.7, Mot. =   0.881 deg/day, Amor

 (480934)     Pos = (04 50, +24.1), V = 20.6, El. = 145.8, 1-sig unc. =      0.0", b = -12.9, Mot. =   0.239 deg/day, Amor, fading

 (482244)     Pos = (11 47, +23.7), V = 20.6, El. = 118.9, 1-sig unc. =      0.0", b = +75.2, Mot. =   0.629 deg/day, Amor, brightening

 2010 WH      Pos = (01 32, +38.8), V = 20.6, El. = 106.0, 1-sig unc. =      0.1", b = -23.4, Mot. =   0.387 deg/day, Amor, fading

 2023 LM2     Pos = (09 25, +26.3), V = 20.6, El. = 151.2, 1-sig unc. =      0.0", b = +44.3, Mot. =   0.758 deg/day, Apollo

 2021 VQ11    Pos = (06 28, -22.1), V = 20.6, El. = 133.9, 1-sig unc. =      0.0", b = -14.9, Mot. =   0.253 deg/day, Amor, fading

 (809757)     Pos = (07 36, +11.3), V = 20.6, El. = 168.4, 1-sig unc. =      0.0", b = +15.0, Mot. =   0.434 deg/day, Apollo, fading

 2016 AP8     Pos = (06 57, +23.7), V = 20.6, El. = 174.5, 1-sig unc. =      0.2", b = +11.7, Mot. =   0.731 deg/day, Apollo, fading

 (137175)     Pos = (15 06, -03.4), V = 20.6, El. =  64.1, 1-sig unc. =      0.0", b = +45.4, Mot. =   0.739 deg/day, Apollo, fading

 (457768)     Pos = (00 03, +12.2), V = 20.6, El. =  77.4, 1-sig unc. =      0.0", b = -48.9, Mot. =   1.192 deg/day, PHA, Apollo, fading

 (204232)     Pos = (09 26, +32.8), V = 20.6, El. = 150.2, 1-sig unc. =      0.0", b = +45.5, Mot. =   0.507 deg/day, Apollo, brightening

 2019 UA6     Pos = (06 18, +42.7), V = 20.6, El. = 155.8, 1-sig unc. =      0.0", b = +12.4, Mot. =   0.399 deg/day, Amor, fading

 2021 BY2     Pos = (09 08, +16.1), V = 20.6, El. = 153.7, 1-sig unc. =      0.4", b = +37.4, Mot. =   1.319 deg/day, Apollo, brightening

  (27346)     Pos = (14 31, +15.9), V = 20.6, El. =  80.3, 1-sig unc. =      0.0", b = +64.4, Mot. =   0.369 deg/day, Amor

 2019 UD8     Pos = (04 30, -02.7), V = 20.6, El. = 131.8, 1-sig unc. =      0.1", b = -32.3, Mot. =   1.298 deg/day, Amor, brightening

 (208617)     Pos = (01 52, +26.9), V = 20.6, El. = 106.7, 1-sig unc. =      0.0", b = -34.0, Mot. =   0.464 deg/day, Apollo, fading

 2016 BN1     Pos = (08 31, -22.1), V = 20.6, El. = 132.4, 1-sig unc. =      0.1", b = +10.2, Mot. =   1.179 deg/day, Amor

 (859061)     Pos = (00 50, +24.0), V = 20.6, El. =  92.6, 1-sig unc. =      0.1", b = -38.9, Mot. =   0.606 deg/day, Amor, fading

 2025 TZ      Pos = (09 35, +14.1), V = 20.6, El. = 146.9, 1-sig unc. =      0.0", b = +42.6, Mot. =   0.790 deg/day, 1-opp Amor, fading

 (354713)     Pos = (13 12, +10.2), V = 20.6, El. =  95.5, 1-sig unc. =      0.1", b = +72.4, Mot. =   0.383 deg/day, Amor, brightening

  (10302)     Pos = (03 37, +16.5), V = 20.6, El. = 127.4, 1-sig unc. =      0.0", b = -30.8, Mot. =   0.153 deg/day, Amor, fading

 2001 SH276   Pos = (03 31, -35.9), V = 20.6, El. = 100.7, 1-sig unc. =      0.0", b = -55.1, Mot. =   0.324 deg/day, Apollo, fading

   (9172)     Pos = (09 00, +13.4), V = 20.6, El. = 154.6, 1-sig unc. =      0.0", b = +34.4, Mot. =   0.267 deg/day, Amor, brightening

 (152952)     Pos = (14 53, +46.7), V = 20.6, El. =  91.4, 1-sig unc. =      0.0", b = +59.2, Mot. =   0.130 deg/day, Amor, brightening

   (8013)     Pos = (14 05, -04.6), V = 20.6, El. =  77.7, 1-sig unc. =      0.0", b = +53.5, Mot. =   0.686 deg/day, Amor, brightening

 2017 RM15    Pos = (02 05, +21.4), V = 20.6, El. = 107.9, 1-sig unc. =      0.1", b = -38.4, Mot. =   0.845 deg/day, Amor, fading

 (154347)     Pos = (11 26, +04.0), V = 20.6, El. = 117.7, 1-sig unc. =      0.0", b = +59.2, Mot. =   0.130 deg/day, Apollo, brightening

 (159533)     Pos = (10 54, +27.7), V = 20.6, El. = 131.5, 1-sig unc. =      0.0", b = +64.0, Mot. =   0.230 deg/day, Amor, brightening

 (837134)     Pos = (11 58, +12.8), V = 20.6, El. = 113.6, 1-sig unc. =      0.0", b = +70.9, Mot. =   0.115 deg/day, Apollo, brightening

 (530974)     Pos = (07 22, +12.8), V = 20.6, El. = 170.6, 1-sig unc. =      0.0", b = +12.6, Mot. =   0.512 deg/day, PHA, Apollo, brightening

 2007 YZ      Pos = (02 07, +07.1), V = 20.6, El. = 103.5, 1-sig unc. =      0.0", b = -51.2, Mot. =   0.442 deg/day, Apollo, fading

 (164216)     Pos = (10 37, -05.9), V = 20.6, El. = 124.2, 1-sig unc. =      0.0", b = +43.6, Mot. =   0.270 deg/day, PHA, Apollo, brightening

 (164207)     Pos = (13 03, +74.3), V = 20.6, El. = 112.5, 1-sig unc. =      0.0", b = +42.8, Mot. =   0.508 deg/day, PHA, Apollo, fading

 2024 YP9     Pos = (09 08, -47.9), V = 20.6, El. = 105.8, 1-sig unc. =      0.1", b = -00.2, Mot. =   1.160 deg/day, 1-opp Apollo, fading

 2008 WB59    Pos = (01 06, +23.0), V = 20.6, El. =  95.6, 1-sig unc. =      0.0", b = -39.8, Mot. =   0.533 deg/day, Apollo, fading

 2008 CE5     Pos = (10 54, -31.0), V = 20.6, El. = 106.1, 1-sig unc. =      0.7", b = +25.4, Mot. =   2.136 deg/day, Apollo, brightening

 (665429)     Pos = (02 01, +01.4), V = 20.6, El. = 100.0, 1-sig unc. =      0.0", b = -57.0, Mot. =   0.613 deg/day, Amor

 2025 VB7     Pos = (08 47, +47.6), V = 20.6, El. = 149.1, 1-sig unc. =      0.1", b = +38.9, Mot. =   0.919 deg/day, 1-opp Amor, fading

 2025 VG1     Pos = (04 35, +32.8), V = 20.6, El. = 142.2, 1-sig unc. =      0.1", b = -10.0, Mot. =   1.048 deg/day, 1-opp Amor, fading

 (516155)     Pos = (02 53, +41.0), V = 20.6, El. = 121.6, 1-sig unc. =      0.0", b = -16.3, Mot. =   0.330 deg/day, PHA, Apollo, fading

   (5189)     Pos = (05 18, +18.9), V = 20.7, El. = 151.4, 1-sig unc. =      0.0", b = -10.7, Mot. =   0.294 deg/day, PHA, Apollo, fading

 2018 LA16    Pos = (07 41, +29.4), V = 20.7, El. = 171.4, 1-sig unc. =      0.2", b = +22.9, Mot. =   0.394 deg/day, Amor, brightening

 2017 BE29    Pos = (05 50, +48.3), V = 20.7, El. = 148.4, 1-sig unc. =      0.0", b = +10.6, Mot. =   0.742 deg/day, Amor

  (39796)     Pos = (09 38, +06.2), V = 20.7, El. = 142.9, 1-sig unc. =      0.0", b = +39.7, Mot. =   0.183 deg/day, Amor, brightening

 (831325)     Pos = (02 29, +02.0), V = 20.7, El. = 106.8, 1-sig unc. =      0.0", b = -52.6, Mot. =   0.732 deg/day, Amor, fading

 (276786)     Pos = (08 23, +11.2), V = 20.7, El. = 161.3, 1-sig unc. =      0.0", b = +25.4, Mot. =   0.335 deg/day, Amor, brightening

 2025 WB14    Pos = (10 15, +36.5), V = 20.7, El. = 139.6, 1-sig unc. =      0.5", b = +55.7, Mot. =   1.171 deg/day, 1-opp Amor, fading

 (285110)     Pos = (08 11, +39.7), V = 20.7, El. = 159.4, 1-sig unc. =      0.0", b = +31.6, Mot. =   0.369 deg/day, Amor, fading

 (159459)     Pos = (11 57, -34.5), V = 20.7, El. =  93.1, 1-sig unc. =      0.0", b = +27.1, Mot. =   0.106 deg/day, Apollo

 (159495)     Pos = (00 39, +08.9), V = 20.7, El. =  84.0, 1-sig unc. =      0.0", b = -53.8, Mot. =   0.697 deg/day, Amor, fading

 (314079)     Pos = (08 27, +59.5), V = 20.7, El. = 140.8, 1-sig unc. =      0.0", b = +35.2, Mot. =   0.700 deg/day, Apollo

 (237442)     Pos = (04 08, -08.4), V = 20.7, El. = 124.0, 1-sig unc. =      0.0", b = -40.0, Mot. =   0.109 deg/day, Amor, fading

 2025 XJ4     Pos = (07 30, -27.6), V = 20.7, El. = 130.2, 1-sig unc. =      0.2", b = -04.5, Mot. =   2.207 deg/day, 1-opp Amor, fading

 (468583)     Pos = (09 14, +04.4), V = 20.7, El. = 147.1, 1-sig unc. =      0.0", b = +33.5, Mot. =   0.370 deg/day, Apollo, brightening

 2025 XW4     Pos = (07 12, -10.9), V = 20.7, El. = 146.9, 1-sig unc. =      0.8", b = -00.5, Mot. =   1.191 deg/day, 1-opp Amor, fading

 (483506)     Pos = (07 47, +15.2), V = 20.7, El. = 170.5, 1-sig unc. =      0.0", b = +18.9, Mot. =   0.436 deg/day, Amor, fading

 2025 XU6     Pos = (12 15, +28.1), V = 20.7, El. = 113.8, 1-sig unc. =      1.8", b = +81.9, Mot. =   0.749 deg/day, 1-opp Amor, brightening

 (490354)     Pos = (07 15, +23.0), V = 20.7, El. = 178.8, 1-sig unc. =      0.0", b = +15.3, Mot. =   0.674 deg/day, PHA, Apollo, fading

  (12923)     Pos = (11 01, +03.8), V = 20.7, El. = 123.4, 1-sig unc. =      0.0", b = +54.8, Mot. =   0.057 deg/day, PHA, Apollo, brightening

 (839546)     Pos = (01 57, +24.8), V = 20.7, El. = 107.3, 1-sig unc. =      0.0", b = -35.7, Mot. =   0.696 deg/day, Amor, fading

  (20429)     Pos = (02 42, +17.6), V = 20.7, El. = 115.2, 1-sig unc. =      0.0", b = -37.8, Mot. =   0.058 deg/day, Apollo, fading

 (606438)     Pos = (06 47, +73.8), V = 20.7, El. = 128.2, 1-sig unc. =      0.0", b = +25.7, Mot. =   0.336 deg/day, Amor, fading

 2025 AK2     Pos = (10 30, -32.5), V = 20.7, El. = 108.9, 1-sig unc. =      0.2", b = +21.5, Mot. =   1.601 deg/day, PHA, Aten, fading

 2012 BJ86    Pos = (12 30, -03.1), V = 20.7, El. = 100.1, 1-sig unc. =      0.1", b = +59.3, Mot. =   1.362 deg/day, PHA, Apollo, brightening

 (749996)     Pos = (08 03, +52.9), V = 20.7, El. = 148.2, 1-sig unc. =      0.0", b = +32.0, Mot. =   0.344 deg/day, Amor, brightening

 (333358)     Pos = (08 51, -12.7), V = 20.7, El. = 138.4, 1-sig unc. =      0.0", b = +19.5, Mot. =   0.436 deg/day, Amor

  (20826)     Pos = (23 28, +11.2), V = 20.7, El. =  69.5, 1-sig unc. =      0.0", b = -46.7, Mot. =   0.144 deg/day, Apollo

 2014 QQ434   Pos = (10 37, -37.8), V = 20.7, El. = 104.2, 1-sig unc. =     18.8", b = +17.9, Mot. =   0.680 deg/day, Apollo, fading

 2013 BF27    Pos = (10 28, +03.3), V = 20.7, El. = 130.7, 1-sig unc. =      4.7", b = +48.2, Mot. =   0.927 deg/day, Amor, brightening

 2025 YR1     Pos = (06 38, +40.6), V = 20.7, El. = 159.6, 1-sig unc. =      0.0", b = +15.0, Mot. =   1.646 deg/day, 1-opp Aten

 2021 YM      Pos = (09 14, +22.3), V = 20.8, El. = 153.5, 1-sig unc. =      0.0", b = +40.8, Mot. =   0.553 deg/day, Apollo, brightening

 2025 XG4     Pos = (10 46, +20.2), V = 20.8, El. = 132.0, 1-sig unc. =      8.5", b = +60.6, Mot. =   1.582 deg/day, 1-opp Amor, brightening

 2025 TZ2     Pos = (10 17, +33.1), V = 20.8, El. = 139.6, 1-sig unc. =      0.1", b = +56.2, Mot. =   0.235 deg/day, 1-opp Amor, fading

 2020 RA81    Pos = (04 37, -35.6), V = 20.8, El. = 110.6, 1-sig unc. =      1.2", b = -41.7, Mot. =   0.633 deg/day, Amor

 2025 YP      Pos = (07 30, +10.1), V = 20.8, El. = 167.6, 1-sig unc. =      0.5", b = +13.2, Mot. =   0.636 deg/day, 1-opp Apollo, fading

 2025 WL8     Pos = (11 18, +47.9), V = 20.8, El. = 126.5, 1-sig unc. =      0.2", b = +62.4, Mot. =   1.020 deg/day, 1-opp Amor, fading

 2025 NK      Pos = (09 29, -06.7), V = 20.8, El. = 137.0, 1-sig unc. =      0.1", b = +30.6, Mot. =   1.474 deg/day, 1-opp Apollo, fading

 2020 YR1     Pos = (00 39, +83.9), V = 20.8, El. = 111.0, 1-sig unc. =      2.7", b = +21.1, Mot. =   1.865 deg/day, Aten, fading

 2025 NA2     Pos = (05 44, -08.8), V = 20.8, El. = 141.3, 1-sig unc. =      0.1", b = -18.8, Mot. =   0.493 deg/day, Amor, fading

 (162273)     Pos = (01 39, +08.6), V = 20.8, El. =  97.6, 1-sig unc. =      0.0", b = -52.5, Mot. =   0.339 deg/day, Amor, fading

 (428086)     Pos = (06 14, +28.7), V = 20.8, El. = 163.9, 1-sig unc. =      0.0", b = +05.3, Mot. =   0.260 deg/day, Amor, fading

 (606367)     Pos = (03 19, +57.4), V = 20.8, El. = 124.5, 1-sig unc. =      0.0", b = +00.1, Mot. =   0.276 deg/day, Apollo, fading

 2008 XB1     Pos = (08 13, +03.5), V = 20.8, El. = 157.4, 1-sig unc. =      0.0", b = +19.6, Mot. =   0.728 deg/day, PHA, Apollo, fading

   (5836)     Pos = (09 32, +04.2), V = 20.8, El. = 143.3, 1-sig unc. =      0.0", b = +37.2, Mot. =   0.166 deg/day, Amor, brightening

 2011 VP12    Pos = (07 07, +00.4), V = 20.8, El. = 158.0, 1-sig unc. =      0.1", b = +03.6, Mot. =   0.251 deg/day, Amor, fading

 2009 SG18    Pos = (08 14, -68.2), V = 20.8, El. =  89.1, 1-sig unc. =      0.0", b = -17.9, Mot. =   0.359 deg/day, PHA, Apollo, fading

 (216115)     Pos = (00 19, -01.0), V = 20.8, El. =  75.7, 1-sig unc. =      0.0", b = -62.8, Mot. =   0.262 deg/day, PHA, Apollo, brightening

 (478574)     Pos = (15 02, -00.9), V = 20.8, El. =  66.0, 1-sig unc. =      0.1", b = +47.9, Mot. =   1.109 deg/day, PHA, Apollo, brightening

 (511243)     Pos = (10 39, +19.7), V = 20.8, El. = 133.5, 1-sig unc. =      0.0", b = +58.9, Mot. =   0.053 deg/day, Amor, brightening

 (538085)     Pos = (10 13, -33.7), V = 20.8, El. = 110.5, 1-sig unc. =      0.0", b = +18.5, Mot. =   0.418 deg/day, Apollo, brightening

 (333908)     Pos = (06 33, -43.0), V = 20.8, El. = 113.9, 1-sig unc. =      0.0", b = -21.3, Mot. =   0.276 deg/day, Amor, fading

   (8176)     Pos = (07 22, +65.8), V = 20.8, El. = 136.4, 1-sig unc. =      0.0", b = +27.7, Mot. =   0.409 deg/day, Apollo, fading

   (8201)     Pos = (02 10, +01.9), V = 20.8, El. = 102.3, 1-sig unc. =      0.0", b = -55.4, Mot. =   0.054 deg/day, Apollo

 (159857)     Pos = (11 33, -25.1), V = 20.8, El. = 102.4, 1-sig unc. =      0.0", b = +34.5, Mot. =   0.119 deg/day, PHA, Apollo, brightening

 (106589)     Pos = (01 09, -06.2), V = 20.8, El. =  85.2, 1-sig unc. =      0.0", b = -68.7, Mot. =   0.164 deg/day, Apollo

 (386298)     Pos = (06 59, +48.6), V = 20.8, El. = 153.3, 1-sig unc. =      0.0", b = +21.2, Mot. =   0.553 deg/day, Amor, fading

 2013 XS3     Pos = (07 20, -42.2), V = 20.8, El. = 115.7, 1-sig unc. =      0.5", b = -13.0, Mot. =   1.249 deg/day, Apollo, fading

  (38091)     Pos = (02 15, +12.1), V = 20.8, El. = 107.2, 1-sig unc. =      0.0", b = -45.8, Mot. =   0.159 deg/day, Amor, fading

  (15817)     Pos = (13 14, -10.9), V = 20.8, El. =  87.2, 1-sig unc. =      0.0", b = +51.6, Mot. =   0.724 deg/day, Amor, brightening

  (13553)     Pos = (05 33, +14.8), V = 20.8, El. = 153.8, 1-sig unc. =      0.0", b = -09.8, Mot. =   0.227 deg/day, Amor, fading

   (4596)     Pos = (12 45, +04.1), V = 20.8, El. =  99.5, 1-sig unc. =      0.0", b = +67.0, Mot. =   0.238 deg/day, Amor, brightening

 (752877)     Pos = (05 24, -01.2), V = 20.8, El. = 143.3, 1-sig unc. =      0.0", b = -20.0, Mot. =   0.531 deg/day, Apollo, fading

 (137799)     Pos = (13 18, +02.2), V = 20.8, El. =  91.1, 1-sig unc. =      0.0", b = +64.2, Mot. =   0.718 deg/day, Amor, brightening

 (143404)     Pos = (23 40, -03.3), V = 20.8, El. =  65.9, 1-sig unc. =      0.0", b = -60.7, Mot. =   0.664 deg/day, PHA, Apollo, fading

 (414429)     Pos = (14 42, +08.3), V = 20.9, El. =  74.5, 1-sig unc. =      0.0", b = +57.8, Mot. =   0.940 deg/day, Apollo, brightening

 2025 YV4     Pos = (06 51, +19.1), V = 20.9, El. = 172.7, 1-sig unc. =      2.5", b = +08.5, Mot. =   2.171 deg/day, 1-opp Apollo, fading

 2025 YT1     Pos = (01 58, +05.0), V = 20.9, El. = 100.7, 1-sig unc. =      0.2", b = -54.1, Mot. =   2.140 deg/day, 1-opp Amor, fading

 (666673)     Pos = (03 41, +73.7), V = 20.9, El. = 120.8, 1-sig unc. =      0.0", b = +14.7, Mot. =   0.305 deg/day, Amor, fading

  (19764)     Pos = (10 10, +10.1), V = 20.9, El. = 137.4, 1-sig unc. =      0.0", b = +48.5, Mot. =   0.125 deg/day, Amor, brightening

 (488450)     Pos = (12 59, +44.3), V = 20.9, El. = 108.9, 1-sig unc. =      0.1", b = +72.8, Mot. =   0.346 deg/day, Amor, brightening

 (426082)     Pos = (03 05, -23.2), V = 20.9, El. = 103.2, 1-sig unc. =      0.1", b = -59.6, Mot. =   0.859 deg/day, Amor

 2025 XM5     Pos = (02 45, +42.5), V = 20.9, El. = 120.2, 1-sig unc. =      0.2", b = -15.7, Mot. =   1.328 deg/day, 1-opp Apollo, fading

 (177614)     Pos = (08 12, +21.7), V = 20.9, El. = 167.9, 1-sig unc. =      0.0", b = +27.0, Mot. =   0.353 deg/day, PHA, Apollo, brightening

 (826946)     Pos = (04 44, +17.5), V = 20.9, El. = 143.1, 1-sig unc. =      0.0", b = -18.2, Mot. =   0.108 deg/day, Amor, fading

 2025 XH4     Pos = (08 19, +32.0), V = 20.9, El. = 163.5, 1-sig unc. =      0.2", b = +31.6, Mot. =   0.483 deg/day, 1-opp Amor, brightening

 2025 XB5     Pos = (08 42, +00.5), V = 20.9, El. = 150.5, 1-sig unc. =      4.1", b = +24.5, Mot. =   0.987 deg/day, 1-opp Amor, brightening

 (438430)     Pos = (14 32, +25.1), V = 20.9, El. =  84.2, 1-sig unc. =      0.1", b = +67.4, Mot. =   1.169 deg/day, Amor, fading

 2004 ME6     Pos = (06 36, +18.2), V = 20.9, El. = 169.2, 1-sig unc. =      0.0", b = +05.0, Mot. =   0.540 deg/day, Apollo, fading

 2012 AA11    Pos = (13 31, +05.9), V = 20.9, El. =  89.5, 1-sig unc. =      0.2", b = +66.7, Mot. =   1.455 deg/day, PHA, Amor

 2022 OF6     Pos = (04 35, +12.7), V = 20.9, El. = 139.7, 1-sig unc. =      0.0", b = -22.8, Mot. =   0.300 deg/day, Amor, fading

 (329275)     Pos = (05 47, -03.7), V = 20.9, El. = 145.7, 1-sig unc. =      0.0", b = -16.0, Mot. =   0.320 deg/day, Apollo, fading

   (6050)     Pos = (13 47, -04.3), V = 20.9, El. =  82.0, 1-sig unc. =      0.0", b = +55.8, Mot. =   0.171 deg/day, Amor, brightening

   (5869)     Pos = (14 32, -25.8), V = 20.9, El. =  64.9, 1-sig unc. =      0.0", b = +31.8, Mot. =   0.510 deg/day, Amor

 2022 AZ7     Pos = (13 24, -14.3), V = 20.9, El. =  83.7, 1-sig unc. =      0.6", b = +47.8, Mot. =   0.757 deg/day, Apollo, brightening

 2020 VZ5     Pos = (04 12, -30.4), V = 20.9, El. = 110.8, 1-sig unc. =      0.0", b = -46.3, Mot. =   0.260 deg/day, PHA, Aten, fading

 2017 YT7     Pos = (05 53, +40.6), V = 20.9, El. = 154.0, 1-sig unc. =      0.1", b = +07.2, Mot. =   0.558 deg/day, Apollo, fading

 2018 AH      Pos = (11 49, +23.9), V = 20.9, El. = 118.6, 1-sig unc. =      0.0", b = +75.6, Mot. =   1.268 deg/day, Apollo, fading

 2013 WW44    Pos = (03 28, +55.0), V = 20.9, El. = 126.2, 1-sig unc. =      0.1", b = -01.3, Mot. =   0.649 deg/day, Apollo, fading

 2021 PS8     Pos = (02 59, +11.5), V = 20.9, El. = 117.1, 1-sig unc. =      0.0", b = -40.6, Mot. =   0.995 deg/day, Amor, fading

 2016 PO39    Pos = (14 21, +00.1), V = 20.9, El. =  75.9, 1-sig unc. =      0.4", b = +55.5, Mot. =   1.641 deg/day, PHA, Apollo

 2025 UL8     Pos = (07 01, +01.1), V = 20.9, El. = 158.4, 1-sig unc. =      0.1", b = +02.6, Mot. =   0.378 deg/day, Amor, fading

 (523607)     Pos = (07 54, +37.1), V = 20.9, El. = 163.4, 1-sig unc. =      0.0", b = +27.7, Mot. =   0.346 deg/day, Amor, brightening

 2025 UU3     Pos = (03 29, -29.2), V = 21.0, El. = 104.3, 1-sig unc. =      0.1", b = -55.4, Mot. =   0.613 deg/day, 1-opp Amor, fading

 (277958)     Pos = (02 16, +27.2), V = 21.0, El. = 111.9, 1-sig unc. =      0.0", b = -32.0, Mot. =   0.118 deg/day, Apollo, fading

 2016 CS246   Pos = (11 55, -14.2), V = 21.0, El. = 103.3, 1-sig unc. =      0.1", b = +46.5, Mot. =   1.218 deg/day, Amor, brightening

 2013 DL1     Pos = (11 02, -63.1), V = 21.0, El. =  84.2, 1-sig unc. =      1.0", b = -02.8, Mot. =   2.485 deg/day, Aten, brightening

 2025 XH3     Pos = (09 28, +51.2), V = 21.0, El. = 141.8, 1-sig unc. =      0.1", b = +45.0, Mot. =   0.799 deg/day, 1-opp Amor, brightening

 2025 YC5     Pos = (12 37, +06.0), V = 21.0, El. = 102.1, 1-sig unc. =      7.6", b = +68.6, Mot. =   0.840 deg/day, 1-opp Amor, brightening

 (405189)     Pos = (10 58, -06.7), V = 21.0, El. = 119.2, 1-sig unc. =      0.1", b = +46.4, Mot. =   0.544 deg/day, Amor, brightening

 2017 SR32    Pos = (02 03, -44.1), V = 21.0, El. =  82.1, 1-sig unc. =      0.1", b = -67.8, Mot. =   0.782 deg/day, Amor, brightening

 (238063)     Pos = (13 38, -10.3), V = 21.0, El. =  81.8, 1-sig unc. =      0.0", b = +50.8, Mot. =   0.347 deg/day, Apollo, brightening

 (530940)     Pos = (02 14, +24.5), V = 21.0, El. = 110.8, 1-sig unc. =      0.0", b = -34.7, Mot. =   0.284 deg/day, Amor, fading

  (86450)     Pos = (13 55, +02.5), V = 21.0, El. =  82.8, 1-sig unc. =      0.0", b = +61.0, Mot. =   0.600 deg/day, Aten, brightening

 2016 JU33    Pos = (14 41, -17.8), V = 21.0, El. =  65.1, 1-sig unc. =      0.0", b = +37.9, Mot. =   2.089 deg/day, PHA, Apollo, fading

 (361538)     Pos = (05 41, -58.0), V = 21.0, El. =  97.2, 1-sig unc. =      0.0", b = -31.9, Mot. =   0.336 deg/day, Apollo, fading

 2025 YW8     Pos = (08 02, +35.9), V = 21.0, El. = 163.4, 1-sig unc. =    678.5", b = +29.1, Mot. =   1.181 deg/day, 1-opp Apollo, brightening

 (458062)     Pos = (14 06, -49.7), V = 21.0, El. =  65.9, 1-sig unc. =      0.1", b = +11.4, Mot. =   1.397 deg/day, Apollo, brightening

 2001 XQ31    Pos = (01 57, +09.6), V = 21.0, El. = 102.3, 1-sig unc. =      0.4", b = -49.9, Mot. =   1.185 deg/day, Amor, fading

 (470909)     Pos = (11 48, -17.9), V = 21.0, El. = 103.2, 1-sig unc. =      0.5", b = +42.4, Mot. =   0.207 deg/day, Amor, brightening

 (424482)     Pos = (03 15, +13.5), V = 21.0, El. = 121.6, 1-sig unc. =      0.0", b = -36.4, Mot. =   0.222 deg/day, PHA, Apollo, fading

 2025 QO2     Pos = (09 19, -02.5), V = 21.0, El. = 141.8, 1-sig unc. =      0.1", b = +30.8, Mot. =   0.473 deg/day, 1-opp Amor, fading

 (316695)     Pos = (10 32, +57.2), V = 21.0, El. = 130.7, 1-sig unc. =      0.0", b = +51.2, Mot. =   0.302 deg/day, Amor

 (358471)     Pos = (13 35, -03.3), V = 21.0, El. =  85.3, 1-sig unc. =      0.0", b = +57.8, Mot. =   0.212 deg/day, PHA, Apollo

 2025 XU5     Pos = (07 15, +36.3), V = 21.0, El. = 165.8, 1-sig unc. =      2.3", b = +20.1, Mot. =   2.609 deg/day, 1-opp Amor, fading

 2022 DC5     Pos = (07 42, +07.9), V = 21.0, El. = 164.8, 1-sig unc. =      0.0", b = +14.7, Mot. =   0.873 deg/day, Apollo, fading

 (414903)     Pos = (14 08, -18.3), V = 21.0, El. =  72.3, 1-sig unc. =      0.0", b = +40.9, Mot. =   0.613 deg/day, Amor, brightening

 (413216)     Pos = (10 00, +34.2), V = 21.0, El. = 143.0, 1-sig unc. =      0.0", b = +52.7, Mot. =   0.434 deg/day, Amor, brightening

 2025 YC2     Pos = (06 15, +23.8), V = 21.0, El. = 165.0, 1-sig unc. =      0.5", b = +03.1, Mot. =   0.763 deg/day, 1-opp Apollo, fading

 (471083)     Pos = (03 02, +16.5), V = 21.0, El. = 119.5, 1-sig unc. =      0.0", b = -36.0, Mot. =   0.360 deg/day, Apollo, fading

 (310737)     Pos = (10 57, +07.8), V = 21.0, El. = 125.8, 1-sig unc. =      0.0", b = +56.8, Mot. =   0.144 deg/day, Amor, brightening

 (474532)     Pos = (00 25, +16.6), V = 21.0, El. =  84.1, 1-sig unc. =      0.0", b = -45.8, Mot. =   0.598 deg/day, Amor

 (155340)     Pos = (04 53, +21.6), V = 21.0, El. = 146.0, 1-sig unc. =      0.0", b = -14.0, Mot. =   0.205 deg/day, Amor, fading
"""


In [10]:
# make list of object names by copying from NEA Observation Planning Aid output

ids = [int(m.group(1)) for m in re.finditer(r'(?m)^\s*\((\d+)\)', raw)]

# set obs time
utc = Time("2026-01-15 02:00:00", scale='utc')

# get target altitudes
els = {}
apmags = {}
hour_angles = {}
for n, name in tqdm(enumerate(ids), total=len(ids)):
    obj = Horizons(name, location=954, epochs=[utc.tdb.jd])
    tab = obj.ephemerides()
    els[name] = float(tab['EL'][0])
    hour_angles[name] = float(tab['hour_angle'][0])
    apmags[name] = float(tab['V'][0])

100%|██████████| 310/310 [00:15<00:00, 20.64it/s]


In [ ]:
cutoff = sorted(els.values())[-5]

for name, alt in alts.items():
    if alt >= cutoff:
        print(f"{name} will be at alt {alt} at {utc}. It is mag {apmags[name]}")